![Banner](images/banner.png)

# JSON

Documentation reference link: [Working with the JSON Data Type](https://cx-oracle.readthedocs.io/en/latest/user_guide/json_data_type.html)

In [1]:
import cx_Oracle
import os
import platform
import time

if platform.system() == 'Darwin':
    cx_Oracle.init_oracle_client(lib_dir=os.environ.get("HOME")+"/instantclient_19_8")
elif platform.system() == 'Windows':
     cx_Oracle.init_oracle_client(lib_dir=r"C:\oracle\instantclient_21_3")

In [12]:
un = "pythondemo"
pw = "welcome"
cs = "localhost/orclpdb1"

connection = cx_Oracle.connect(user=un, password=pw, dsn=cs)

### JSON Storage:

- Oracle Database 12c uses LOB storage

- Oracle Database 21c introduces OSON storage: new optimized native binary format

**Careful coding is required for apps that run in a mixed version environment**

The first JSON example assumes you are using Oracle Client 21c and Oracle Database 21c.

In [5]:
with connection.cursor() as cursor:
    try:
        cursor.execute("drop table customers")
    except:
        ;
    cursor.execute("create table customers (k number, json_data json)")

With 21c, you can bind Python objects directly to the JSON column:

In [6]:
import datetime

json_data = [
    2.78,
    True,
    'Ocean Beach',
    b'Some bytes',
    {'keyA': 1, 'KeyB': 'Melbourne'},
    datetime.date.today()
]

with connection.cursor() as cursor:
    cursor.setinputsizes(cx_Oracle.DB_TYPE_JSON)
    cursor.execute("insert into customers (k, json_data) values (1, :jbv)", [json_data])
    
print("Done")

DatabaseError: DPI-1050: Oracle Client library is at version 19.8 but version 21.0 or higher is needed

Querying returns the JSON in a familiar Python data structure:

In [7]:
with connection.cursor() as cursor:
    for row, in cursor.execute("select c.json_data from customers c where k = 1"):
        print(row)
        
# With Oracle Client and Oracle Database 21c, this gives:
# [Decimal('2.78'), True, 'Ocean Beach', b'Some bytes', {'keyA': Decimal('1'), 'KeyB': 'Melbourne'}, datetime.datetime(2022, 3, 4, 0, 0)]

If you don't have 21c, then you can still easily work with JSON.  Store it using BLOB and work with JSON strings. The Python "json" package can be used with many types:

In [10]:
import json

with connection.cursor() as cursor:
    try:
        cursor.execute("drop table customersblob")
    except:
        ;
    cursor.execute("""create table customersblob (k number, 
                                              json_data BLOB CHECK (json_data IS JSON)) 
                                                        LOB (json_data) STORE AS (CACHE)""")
 
# INSERT

with connection.cursor() as cursor:
    data = json_data = [
        2.78,
        True,
        'Ocean Beach',
        {'keyA': 1, 'KeyB': 'Melbourne'},
    ]
    cursor.execute("insert into customersblob (k, json_data) values (2, :jbv)", [json.dumps(data)])
 
# FETCH

# For JSON < 1 GB, use an output type handler for performance
def output_type_handler(cursor, name, default_type, size, precision, scale):
    if default_type == cx_Oracle.BLOB:
        return cursor.var(cx_Oracle.LONG_BINARY, arraysize=cursor.arraysize)

connection.outputtypehandler = output_type_handler

with connection.cursor() as cursor:
    for row, in cursor.execute("SELECT c.json_data FROM customersblob c where k = 2"):
        print(row)
    
connection.rollback()   

b'[2.78, true, "Ocean Beach", {"keyA": 1, "KeyB": "Melbourne"}]'


# Simple Oracle Document Access (SODA)

Documentation reference link: [SODA](https://cx-oracle.readthedocs.io/en/latest/user_guide/soda.html)

SODA allows documents to be inserted, queried, and retrieved from Oracle Database using a set of NoSQL-style cx_Oracle methods.  SODA can also be used in other languages like Java, Node.js and PL/SQL.

By default SODA stores JSON documents, but this can be changed.

Connection Pools support a SODA Metadata Cache, which helps performance. This requires Oracle Client 19.11 or later:

In [37]:
r,u,v,i,e = cx_Oracle.clientversion()

if (r > 19 or (r == 19 and u >= 11)):
    pool = cx_Oracle.SessionPool(user=un, password=pw, dsn=cs, 
                             min=4, max=4, increment=0,
                             soda_metadata_cache=True)
    connection = pool.acquire()


The general recommendation for simple SODA usage is to enable autocommit, but be wary of overusing it when you are inserting or updating multiple documents.

In [38]:
connection.autocommit = True

Start by creating the parent object for SODA:

In [39]:
soda = connection.getSodaDatabase()

Clean up so the demo runs identically each time:

In [40]:
# Get a collection
collection = soda.openCollection("mycollection")

# Drop the collection if it already exists
if collection is not None:
    collection.drop()

If you are using Oracle Database 21c with older Oracle Client libraries note the default collection metadata changed in 21c to the improved JSON type.  To avoid compatibility issues, you can explicitly specify the  metadata and use BLOB storage.

In [41]:
# Explicit metadata is used for maximum version interoperability.
# Refer to the documentation.
metadata = {
    "keyColumn": {
        "name": "ID"
    },
    "contentColumn": {
        "name": "JSON_DOCUMENT",
        "sqlType": "BLOB"
    },
    "versionColumn": {
        "name": "VERSION",
        "method": "UUID"
    },
    "lastModifiedColumn": {
        "name": "LAST_MODIFIED"
    },
    "creationTimeColumn": {
        "name": "CREATED_ON"
    }
}

Create a new SODA collection:

In [42]:
collection = soda.openCollection("mycollection")
if collection is None:
    collection = soda.createCollection("mycollection", metadata)

When inserting a document, a system generated key is created by default:

In [43]:
content = {'name': 'Matilda', 'address': {'city': 'Melbourne'}}
doc = collection.insertOneAndGet(content)

key = doc.key
print('The key of the new SODA document is: ', key)

The key of the new SODA document is:  7954D1DC809A4F1CBFDDA72614C87FA2


An operation builder using `find()` allows documents to be acted on. A series of non-terminals can be chained together followed by an action.

In [44]:
doc = collection.find().key(key).getOne() # A SodaDocument

Fetched documents can be accessed in different ways:

In [45]:
content = doc.getContent()                # A JavaScript object
print('SODA document dictionary:')
print(type(content))
print(content)

content = doc.getContentAsString()        # A JSON string
print('SODA document string:')
print(type(content))
print(content)

SODA document dictionary:
<class 'dict'>
{'name': 'Matilda', 'address': {'city': 'Melbourne'}}
SODA document string:
<class 'str'>
{"name": "Matilda", "address": {"city": "Melbourne"}}


To replace document contents, we can find the document matching the previously returned key:

In [46]:
content = {'name': 'Matilda', 'address': {'city': 'Sydney'}}
collection.find().key(key).replaceOne(content)

True

Insert some more documents:

In [47]:
content = [
    {'name': 'Venkat', 'address': {'city': 'Bengaluru'}},
    {'name': 'May', 'address': {'city': 'London'}},
    {'name': 'Sally-Ann', 'address': {'city': 'San Francisco'}}
]
collection.insertMany(content)

print("Done")

Done


Query by Example (QBE) syntax makes it easier to search for documents.

To find all documents with names like 'Ma%':

In [48]:
documents = collection.find().filter({'name': {'$like': 'Ma%'}}).getDocuments()
for d in documents:
    content = d.getContent()
    print(content["name"])

Matilda
May


Count all documents:

In [49]:
c = collection.find().count()
print('Collection has', c, 'documents')

Collection has 4 documents


Remove documents with cities containing 'g':

In [50]:
c = collection.find().filter({'address.city': {'$regex': '.*g.*'}}).remove()
print('Dropped', c, 'documents')

Dropped 1 documents


Drop the collection:

In [51]:
if collection.drop():
    print('Collection was dropped')

Collection was dropped
